In [1]:

import tensorflow as tf
import tensorflow.keras
import keras

In [2]:
vocab = ["square", "rectangle", "pentagon", "hexagon"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [3]:
categories = tf.constant(["heptagon", "hexagon", "square"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([5, 3, 0], dtype=int64)>

In [4]:
cat_one_hot = tf.one_hot(cat_indices, depth= len(vocab)+num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:

embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(6, 2) dtype=float32, numpy=
array([[0.88051474, 0.25234556],
       [0.56814384, 0.8494557 ],
       [0.7522837 , 0.49260974],
       [0.74824846, 0.6619903 ],
       [0.80376303, 0.11152852],
       [0.83025634, 0.5718168 ]], dtype=float32)>

In [6]:
categories = tf.constant(["heptagon", "hexagon", "square"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([5, 3, 0], dtype=int64)>

In [7]:
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.83025634, 0.5718168 ],
       [0.74824846, 0.6619903 ],
       [0.88051474, 0.25234556]], dtype=float32)>

In [8]:
embedding = tf.keras.layers.Embedding(input_dim= len(vocab)+num_oov_buckets, output_dim=embedding_dim)
embedding

In [9]:
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None,)              0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2)            12          lambda[0][0]                     
______________________________________________________________________________________________